In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

# Show3DVolume: CIF File → 3D Electrostatic Potential

Load any crystal structure from a CIF file, place 3D Gaussians at each atom position weighted by atomic number, and visualize all three orthogonal slices interactively.

**Workflow**: CIF → ASE atoms → orthogonalize → supercell → 3D Gaussian potential → Show3DVolume

## 1. Load CIF and Build Supercell

In [ ]:
import numpy as np
import torch
import abtem
from ase.io import read
from ase.data import atomic_numbers, covalent_radii
from quantem.widget import Show3DVolume

device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# Load MoS2 from CIF
atoms = read("../../data/MoS2.cif")
print(f"Formula: {atoms.get_chemical_formula()}")
print(f"Unit cell: {atoms.cell.lengths()}")

# Orthogonalize for abTEM (hexagonal → orthorhombic)
atoms_ortho = abtem.orthogonalize_cell(atoms)
a, b, c = atoms_ortho.cell.lengths()
print(f"Orthorhombic cell: a={a:.3f} Å, b={b:.3f} Å, c={c:.3f} Å")
print(f"Atoms in ortho cell: {len(atoms_ortho)}")

Device: mps
Formula: Mo2S4
Unit cell: [ 3.18543231  3.18543231 13.17839091]
Orthorhombic cell: a=3.185 Å, b=5.517 Å, c=13.178 Å
Atoms in ortho cell: 12


In [ ]:
# Build supercell: 6×6 lateral, 3× along c-axis
slab = atoms_ortho * (6, 6, 3)
cell = slab.cell.lengths()
print(f"Supercell: {len(slab)} atoms")
print(f"Dimensions: {cell[0]:.1f} × {cell[1]:.1f} × {cell[2]:.1f} Å")

Supercell: 1296 atoms
Dimensions: 19.1 × 33.1 × 39.5 Å


## 2. Compute 3D Electrostatic Potential V(r)

Using the **Kirkland parametrization** (Advanced Computing in Electron Microscopy, 2010):

$$V(\mathbf{r}) = \frac{h^2}{2\pi m_e e} \left[ \sum_{i=1}^{3} \frac{\pi a_i}{r} e^{-2\pi\sqrt{b_i}\, r} + \sum_{i=1}^{3} c_i \left(\frac{\pi}{d_i}\right)^{3/2} e^{-\pi^2 r^2 / d_i} \right]$$

Each atom contributes a radial potential with **Lorentzian** (sharp core, $\sim e^{-\beta r}/r$) and **Gaussian** (smooth tail) terms. Parameters $\{a_i, b_i, c_i, d_i\}$ are tabulated per element from quantum scattering calculations.

In [ ]:
from abtem.parametrizations import KirklandParametrization

C_PREFACTOR = 47.878  # h²/(2π m_e e) in V·Å²


def atoms_to_potential(slab, voxel_size=0.2, cutoff=4.0):
    """Compute 3D electrostatic potential V(r) using Kirkland parametrization.

    Parameters
    ----------
    slab : ase.Atoms
        Supercell with atom positions.
    voxel_size : float
        Voxel size in Å.
    cutoff : float
        Radial cutoff in Å (potential negligible beyond this).

    Returns
    -------
    volume : np.ndarray
        3D float32 array (nz, ny, nx) in Volts.
    """
    kirk = KirklandParametrization()
    cell = slab.cell.lengths()
    nx, ny, nz = [int(np.ceil(c / voxel_size)) for c in cell]

    # 1D grid coordinates (in Å)
    gx = torch.arange(nx, dtype=torch.float32, device=device) * voxel_size
    gy = torch.arange(ny, dtype=torch.float32, device=device) * voxel_size
    gz = torch.arange(nz, dtype=torch.float32, device=device) * voxel_size

    volume = torch.zeros((nz, ny, nx), dtype=torch.float32, device=device)

    # Precompute coefficients per element
    symbols = slab.get_chemical_symbols()
    unique_symbols = set(symbols)
    elem_coeffs = {}
    for sym in unique_symbols:
        p = kirk.parameters[sym]  # [[a1,a2,a3], [b1,b2,b3], [c1,c2,c3], [d1,d2,d3]]
        a = torch.tensor(p[0], dtype=torch.float32, device=device)
        b = torch.tensor(p[1], dtype=torch.float32, device=device)
        c = torch.tensor(p[2], dtype=torch.float32, device=device)
        d = torch.tensor(p[3], dtype=torch.float32, device=device)
        # Precompute constants
        alpha = np.pi * a                      # Lorentzian amplitude (3,)
        beta = 2 * np.pi * torch.sqrt(b)       # Lorentzian decay rate (3,)
        delta = c * (np.pi / d) ** 1.5          # Gaussian amplitude (3,)
        gamma = np.pi ** 2 / d                  # Gaussian exponent (3,)
        elem_coeffs[sym] = (alpha, beta, delta, gamma)

    # Accumulate V(r) for each atom
    cutoff_vox = int(np.ceil(cutoff / voxel_size))
    positions = slab.positions  # (N, 3) in Å

    for i, (sym, pos) in enumerate(zip(symbols, positions)):
        alpha, beta, delta, gamma = elem_coeffs[sym]

        # Local voxel range (with cutoff)
        ix0 = max(0, int(pos[0] / voxel_size) - cutoff_vox)
        ix1 = min(nx, int(pos[0] / voxel_size) + cutoff_vox + 1)
        iy0 = max(0, int(pos[1] / voxel_size) - cutoff_vox)
        iy1 = min(ny, int(pos[1] / voxel_size) + cutoff_vox + 1)
        iz0 = max(0, int(pos[2] / voxel_size) - cutoff_vox)
        iz1 = min(nz, int(pos[2] / voxel_size) + cutoff_vox + 1)

        # Local grids
        lx = gx[ix0:ix1] - pos[0]  # (lnx,)
        ly = gy[iy0:iy1] - pos[1]  # (lny,)
        lz = gz[iz0:iz1] - pos[2]  # (lnz,)

        # 3D distance: r[z,y,x] = sqrt(lz² + ly² + lx²)
        r2 = lz[:, None, None] ** 2 + ly[None, :, None] ** 2 + lx[None, None, :] ** 2
        r = torch.sqrt(r2)
        r_safe = torch.clamp(r, min=0.05)  # avoid 1/r singularity

        # Lorentzian terms: Σ α_i/r × exp(-β_i × r)
        V_local = torch.zeros_like(r)
        for j in range(3):
            V_local += (alpha[j] / r_safe) * torch.exp(-beta[j] * r_safe)

        # Gaussian terms: Σ δ_i × exp(-γ_i × r²)
        for j in range(3):
            V_local += delta[j] * torch.exp(-gamma[j] * r2)

        volume[iz0:iz1, iy0:iy1, ix0:ix1] += C_PREFACTOR * V_local

        if (i + 1) % 200 == 0:
            print(f"  {i+1}/{len(positions)} atoms...")

    print(f"  {len(positions)}/{len(positions)} atoms — done")
    return volume.cpu().numpy()


volume = atoms_to_potential(slab, voxel_size=0.2, cutoff=4.0)
print(f"Volume shape: {volume.shape}  (nz × ny × nx)")
print(f"Voxel size: 0.2 Å")
print(f"Value range: [{volume.min():.1f}, {volume.max():.1f}] V")

  200/1296 atoms...
  400/1296 atoms...
  600/1296 atoms...
  800/1296 atoms...
  1000/1296 atoms...
  1200/1296 atoms...
  1296/1296 atoms — done
Volume shape: (198, 166, 96)  (nz × ny × nx)
Voxel size: 0.2 Å
Value range: [0.0, 8153.0] V


## 3. Visualize with Show3DVolume

Three orthogonal slices through V(r). Mo columns (Z=42) appear brighter than S columns (Z=16). The Lorentzian core gives sharp peaks at atom centers; the Gaussian tail extends the potential ~2–3 Å from each nucleus.

In [ ]:
Show3DVolume(
    volume,
    title=f"MoS₂ — Gaussian Potential ({volume.shape[0]}×{volume.shape[1]}×{volume.shape[2]})",
    cmap="inferno",
    pixel_size_angstrom=0.2,
    dim_labels=["Z (c-axis)", "Y", "X"],
)

Show3DVolume(198×166×96, slices=(99,83,48), cmap=inferno)

## 4. Log Scale View

Log scale compresses the dynamic range, making it easier to see the lighter S atoms alongside the heavier Mo atoms.

In [ ]:
Show3DVolume(
    volume,
    title="MoS₂ — Log Scale",
    cmap="viridis",
    pixel_size_angstrom=0.2,
    log_scale=True,
)

Show3DVolume(198×166×96, slices=(99,83,48), cmap=viridis)

## 5. Generic CIF Loader

A reusable function that works with any CIF file. Just change the path and supercell size.

In [ ]:
def cif_to_potential(cif_path, repeat=(6, 6, 3), voxel_size=0.2, cutoff=4.0):
    """Load a CIF file and compute 3D electrostatic potential V(r).

    Parameters
    ----------
    cif_path : str
        Path to the CIF file.
    repeat : tuple of 3 ints
        Supercell repetitions (a, b, c).
    voxel_size : float
        Voxel size in Å.
    cutoff : float
        Radial cutoff in Å.

    Returns
    -------
    volume : np.ndarray
        3D float32 array (nz, ny, nx) in Volts.
    voxel_size : float
        Voxel size in Å.
    formula : str
        Chemical formula.
    """
    atoms = read(cif_path)
    formula = atoms.get_chemical_formula()
    atoms_ortho = abtem.orthogonalize_cell(atoms)
    slab = atoms_ortho * repeat
    volume = atoms_to_potential(slab, voxel_size=voxel_size, cutoff=cutoff)
    return volume, voxel_size, formula


# Use it with a larger supercell:
vol, px, formula = cif_to_potential("../../data/MoS2.cif", repeat=(8, 8, 4))
print(f"{formula}: {vol.shape}, {px} Å/voxel")

Show3DVolume(
    vol,
    title=f"{formula} — Kirkland V(r) — {vol.shape[0]}×{vol.shape[1]}×{vol.shape[2]}",
    cmap="inferno",
    pixel_size_angstrom=px,
)

  200/3072 atoms...
  400/3072 atoms...
  600/3072 atoms...
  800/3072 atoms...
  1000/3072 atoms...
  1200/3072 atoms...
  1400/3072 atoms...
  1600/3072 atoms...
  1800/3072 atoms...
  2000/3072 atoms...
  2200/3072 atoms...
  2400/3072 atoms...
  2600/3072 atoms...
  2800/3072 atoms...
  3000/3072 atoms...
  3072/3072 atoms — done
Mo2S4: (264, 221, 128), 0.2 Å/voxel


Show3DVolume(264×221×128, slices=(132,110,64), cmap=inferno)